In [1]:
import sys
import logging
import sys
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import time
import pickle
import os
 
#logging 
nblog = open("nb.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog
 
get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)
 
%autosave 5
time_start=time.time()
time_start
 
from datetime import datetime, timedelta
from datetime import date
from dateutil.relativedelta import relativedelta

Autosaving every 5 seconds


## Load Data

In [2]:
df = pd.read_csv("credit_card_transactions.csv", index_col = [0])

## EDA

In [4]:
df.cc_num.apply(lambda x : len(str(x))).value_counts()

16    649148
15    215968
14    125985
19    118789
12     91468
13     89206
11      6111
Name: cc_num, dtype: int64

In [5]:
df.is_fraud.value_counts()

0    1289169
1       7506
Name: is_fraud, dtype: int64

In [6]:
df.merchant.value_counts()

fraud_Kilback LLC                       4403
fraud_Cormier LLC                       3649
fraud_Schumm PLC                        3634
fraud_Kuhn LLC                          3510
fraud_Boyer PLC                         3493
                                        ... 
fraud_Douglas, DuBuque and McKenzie      775
fraud_Treutel-King                       775
fraud_Medhurst, Labadie and Gottlieb     759
fraud_Reichert-Weissnat                  753
fraud_Hahn, Douglas and Schowalter       727
Name: merchant, Length: 693, dtype: int64

In [7]:
df[df.is_fraud == 1].merchant.value_counts()

fraud_Rau and Sons                      49
fraud_Cormier LLC                       48
fraud_Kozey-Boehm                       48
fraud_Doyle Ltd                         47
fraud_Vandervort-Funk                   47
                                        ..
fraud_Turner LLC                         1
fraud_Kihn-Fritsch                       1
fraud_Stokes, Christiansen and Sipes     1
fraud_Fadel Inc                          1
fraud_Hills-Olson                        1
Name: merchant, Length: 679, dtype: int64

In [9]:
df['datetime'].min(), df['datetime'].max()

(Timestamp('2019-01-01 00:00:18'), Timestamp('2020-06-21 12:13:37'))

In [222]:
df.groupby(['cc_num','merchant']).agg({'is_fraud' : 'sum',
                                      'datetime' : ['min','max']}). reset_index()

cc_num                          merchant is_fraud  \
                                                                   sum   
0               60416207185               fraud_Abbott-Rogahn        0   
1               60416207185              fraud_Abbott-Steuber        0   
2               60416207185          fraud_Abernathy and Sons        0   
3               60416207185                 fraud_Abshire PLC        0   
4               60416207185  fraud_Adams, Kovacek and Kuhlman        0   
...                     ...                               ...      ...   
479067  4992346398065154184                fraud_Zemlak Group        0   
479068  4992346398065154184  fraud_Zemlak, Tillman and Cremin        0   
479069  4992346398065154184              fraud_Ziemann-Waters        0   
479070  4992346398065154184      fraud_Zieme, Bode and Dooley        0   
479071  4992346398065154184                  fraud_Zulauf LLC        0   

                  datetime                      
                       min                 max  
0      2019-05-05 14:49:41 2020-04-27 18:07:08  
1      2019-05-28 13:28:33 2020-02-14 15:13:50  
2      2019-05-10 12:06:46 2019-08-05 15:13:22  
3      2019-07-01 12:08:53 2019-11-30 21:52:16  
4      2019-05-14 11:07:23 2020-01-05 11:50:41  
...                    ...                 ...  
479067 2019-04-23 00:00:37 2019-12-04 07:17:50  
479068 2019-02-04 15:00:05 2020-02-18 15:21:57  
479069 2019-03-11 18:34:00 2020-03-02 21:04:01  
479070 2019-01-11 03:45:26 2020-06-16 00:30:16  
479071 2019-01-28 23:35:30 2019-01-28 23:35:30  

[479072 rows x 5 columns]

## Functions 

In [252]:
def create_graph(df,sdate_overall, sdate_fraud, edate, card_id = 'cc_num', merch_id = 'merchant', date = 'trans_date_trans_time', label = 'is_fraud'):
    # df['datetime'] = df[date].apply(pd.to_datetime) # Takes a lot of time - remove if already processed
    graph = df[(df.datetime >= sdate_overall) & (df.datetime <= edate)][[card_id, merch_id, 'datetime', 'is_fraud']].reset_index(drop = True)
    cc_fraudwindow = graph[graph[label] == 1][[card_id, 'datetime']].groupby(card_id).agg(first_fraud_date = ('datetime','min')).reset_index()
    cc_fraudwindow = cc_fraudwindow[cc_fraudwindow.first_fraud_date >= sdate_fraud].reset_index(drop = True)
    graph = graph.merge(cc_fraudwindow,
                        on = card_id,
                        how = 'left'
                        )
    return graph

In [257]:
def create_blame_theta(graph, card_id = 'cc_num', merch_id = 'merchant', alpha = 0, beta = 0): #alpha = 0.2, beta = 1000
    breach_graph = graph[graph.datetime <= graph.first_fraud_date].reset_index(drop = True) ## non fraud cards and their links gets removed due to NaT comparisons
    merch_list = list(breach_graph.merchant)
    total = graph[graph.merchant.isin(merch_list)].groupby(merch_id).agg(total_cards = (card_id, 'nunique'))
    fraud = breach_graph.groupby([merch_id]).agg(fraud_cards=(card_id, 'nunique'))    
    
    blame = breach_graph[[card_id,merch_id]].drop_duplicates().reset_index(drop = True)
    blame['blame0'] = 1

    theta = pd.concat([total, fraud], axis=1).fillna(0).astype(int).reset_index()
    theta = theta.merge(blame.groupby(merch_id).agg(sum_blame = ('blame0', 'sum')).reset_index(), on = merch_id, how = 'left')
    theta['theta0'] = (theta['sum_blame']+alpha)/(theta['total_cards']+beta+alpha)
    theta = theta.drop('sum_blame', axis = 1).fillna(0)
    return blame, theta, breach_graph

In [254]:
def update_blame(blame, theta, iteration, card_id = 'cc_num', merch_id = 'merchant', alpha = 0, beta = 0):
    prev = iteration-1
    blame = blame.merge(theta[[merch_id,f'theta{prev}']], on = merch_id, how = 'left')
    blame = blame.merge(blame.groupby(card_id).agg(sum_theta = (f'theta{prev}','sum')).reset_index(), on = card_id, how = 'left')
    blame[f'blame{iteration}'] = blame[f'theta{prev}']/blame['sum_theta']
    blame = blame.drop('sum_theta', axis = 1)

    return blame

In [255]:
def update_theta(blame, theta, iteration, card_id = 'cc_num', merch_id = 'merchant', alpha = 0, beta = 0):
    prev = iteration-1
    theta = theta.merge(blame.groupby(merch_id).agg(sum_blame = (f'blame{iteration}', 'sum')).reset_index(), on = merch_id, how = 'left')
    theta[f'theta{iteration}'] = (theta['sum_blame']+alpha)/(theta['total_cards']+beta+alpha)
    theta = theta.drop('sum_blame', axis = 1).fillna(0)

    return theta

## Sample Runs

In [ ]:
card_id = 'cc_num'
merch_id = 'merchant'
sdate_overall = "2019-01-01"
sdate_fraud = "2019-04-01"
edate = "2019-06-01"

## Optional
# alpha = th.fraud_cards.mean() / th.total_cards.mean() 
# beta = th.total_cards.std()

In [258]:
g = create_graph(df, "2019-01-01","2019-04-01","2019-06-01")

In [260]:
a,b,c = create_blame_theta(g)

In [263]:
b

,merchant,total_cards,fraud_cards,theta0
0,fraud_Abbott-Rogahn,346,31,0.089595
1,fraud_Abbott-Steuber,318,27,0.084906
2,fraud_Abernathy and Sons,304,17,0.055921
3,fraud_Abshire PLC,333,29,0.087087
4,"fraud_Adams, Kovacek and Kuhlman",162,7,0.043210
...,...,...,...,...
688,fraud_Zemlak Group,237,14,0.059072
689,"fraud_Zemlak, Tillman and Cremin",317,19,0.059937
690,fraud_Ziemann-Waters,334,25,0.074850
691,"fraud_Zieme, Bode and Dooley",408,28,0.068627


In [246]:
for i in range(0,10):
    print(f"Iteration {i}")
    if i == 0:
        graph = create_graph(df, "2019-01-01","2019-04-01","2019-06-01")
        b, th = create_blame_theta(graph)
    else:
        b = update_blame(b,th,i)
        th = update_theta(b,th,i)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9


In [247]:
th.sort_values('theta9',ascending = False)

,merchant,total_cards,fraud_cards,theta0,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,theta9
479,fraud_Pfeffer LLC,176,20,0.113636,0.006307,0.012407,0.018183,0.023153,2.709651e-02,3.002343e-02,3.207542e-02,3.343950e-02,3.429451e-02
508,"fraud_Reilly, Heaney and Cole",406,35,0.086207,0.002922,0.005529,0.008034,0.010282,1.217752e-02,1.367907e-02,1.479029e-02,1.554582e-02,1.599904e-02
218,"fraud_Hagenes, Hermann and Stroman",175,22,0.125714,0.000914,0.001353,0.001984,0.002885,4.145644e-03,5.861863e-03,8.107607e-03,1.090408e-02,1.418824e-02
29,fraud_Bauch-Raynor,415,32,0.077108,0.002777,0.005031,0.007000,0.008619,9.875753e-03,1.079592e-02,1.142252e-02,1.180716e-02,1.200270e-02
510,fraud_Rempel Inc,342,24,0.070175,0.003203,0.005415,0.006980,0.008014,8.652929e-03,9.012080e-03,9.178568e-03,9.215318e-03,9.167860e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,"fraud_Swift, Bradtke and Marquardt",175,8,0.045714,0.000103,0.000046,0.000020,0.000009,3.521759e-06,1.421494e-06,5.628536e-07,2.189835e-07,8.379694e-08
674,fraud_Wisozk and Sons,278,12,0.043165,0.000095,0.000042,0.000018,0.000007,3.007791e-06,1.194696e-06,4.637105e-07,1.758102e-07,6.509169e-08
4,"fraud_Adams, Kovacek and Kuhlman",162,7,0.043210,0.000087,0.000036,0.000015,0.000006,2.423975e-06,9.606532e-07,3.746260e-07,1.433932e-07,5.376379e-08
158,fraud_Emmerich-Luettgen,167,7,0.041916,0.000081,0.000032,0.000012,0.000005,1.720557e-06,6.263274e-07,2.229866e-07,7.757463e-08,2.636331e-08


In [248]:
card_id = 'cc_num'
merch_id = 'merchant'
sdate_overall = "2019-01-01"
sdate_fraud = "2019-04-01"
edate = "2019-06-01"

## Optional
# alpha = th.fraud_cards.mean() / th.total_cards.mean() 
# beta = th.total_cards.std()

In [249]:
for i in range(0,10):
    # print(f"Iteration {i}")
    if i == 0:
        graph = create_graph(df, sdate_overall, sdate_fraud, edate)
        b, th = create_blame_theta(graph, card_id = 'cc_num', merch_id = 'merchant', alpha = alpha, beta = beta)
    else:
        b = update_blame(b,th,i, card_id = 'cc_num', merch_id = 'merchant', alpha = alpha, beta = beta)
        th = update_theta(b,th,i, card_id = 'cc_num', merch_id = 'merchant', alpha = alpha, beta = beta)

In [250]:
th.sort_values('theta9',ascending = False)

,merchant,total_cards,fraud_cards,theta0,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,theta9
479,fraud_Pfeffer LLC,176,20,0.072352,0.004002,0.006865,0.008992,0.010510,0.011565,0.012285,0.012767,0.013082,0.013280
508,"fraud_Reilly, Heaney and Cole",406,35,0.069113,0.002387,0.004318,0.006055,0.007538,0.008749,0.009699,0.010418,0.010943,0.011311
29,fraud_Bauch-Raynor,415,32,0.062089,0.002279,0.003966,0.005346,0.006421,0.007223,0.007797,0.008191,0.008448,0.008603
510,fraud_Rempel Inc,342,24,0.054255,0.002510,0.004038,0.005024,0.005631,0.005990,0.006193,0.006300,0.006347,0.006358
239,fraud_Heidenreich PLC,427,29,0.054991,0.002126,0.003298,0.004009,0.004422,0.004654,0.004776,0.004836,0.004860,0.004863
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,"fraud_Cronin, Kshlerin and Weber",303,13,0.032245,0.000107,0.000080,0.000070,0.000066,0.000064,0.000064,0.000063,0.000063,0.000063
421,fraud_Medhurst PLC,369,18,0.038360,0.000134,0.000096,0.000079,0.000070,0.000066,0.000064,0.000063,0.000063,0.000062
674,fraud_Wisozk and Sons,278,12,0.031733,0.000107,0.000081,0.000070,0.000066,0.000064,0.000063,0.000063,0.000062,0.000062
213,"fraud_Gutmann, McLaughlin and Wiza",427,21,0.039829,0.000151,0.000112,0.000089,0.000076,0.000069,0.000065,0.000063,0.000062,0.000061
